# Multi-Agent Debate System Tutorial 🎭

Welcome to an exciting journey into the world of AI agent debates! In this tutorial, we'll learn how to create AI agents that can debate each other on interesting topics.

## What You'll Learn 🎯

In this tutorial, you'll discover:

1. **🤖 Agentic AI**: How to create AI agents with specific roles and personalities
2. **📝 Prompt Engineering**: How to write instructions that make AI behave the way you want
3. **🗣️ Multi-Agent Systems**: How multiple AI agents can interact with each other
4. **⚖️ AI Judging**: How AI can evaluate and judge arguments
5. **🔧 Real Project Structure**: How to organize a real AI project

Let's dive in and create some amazing AI debates! 🚀


## Setup and Prerequisites 🛠️

Before we start creating our debate agents, let's make sure we have everything we need:

### What We Need:
- **Ollama** running on your computer
- **Python packages**: ollama, gTTS (for audio)
- **AI Models**: At least one model like phi3 or llama3

Let's install the required packages:


In [ ]:
# Install required packages
%pip install ollama gtts

# Let's also import everything we'll need
import os
import json
import re
from datetime import datetime
from ollama import chat, list as ollama_list
from typing import Dict, List, Any

print("✅ All packages installed and imported!")


## Part 1: Understanding Debate Topics 📋

First, let's look at what kinds of topics our AI agents can debate about. We'll define some fun and thought-provoking topics that will make for interesting debates!


In [ ]:
# Our debate topics - these are the things our AI agents will argue about!
DEBATE_TOPICS = {
    "pineapple_pizza": "Pineapple is a valid and delicious pizza topping.",
    "four_day_workweek": "The standard work week should be reduced to four days for all industries.",
    "ai_education": "Artificial intelligence should be used to replace human teachers in most classroom settings.",
    "social_media": "Social media platforms have caused more harm than good to modern society.",
    "college_free": "Higher education (college and university) should be provided free of charge to all qualified students.",
}

def list_topics():
    """Show all available debate topics in a nice format"""
    print("🎭 Available Debate Topics:")
    print("=" * 60)
    for i, (key, topic) in enumerate(DEBATE_TOPICS.items(), 1):
        print(f"{i}. {key.replace('_', ' ').title()}: {topic}")
    print("=" * 60)

def get_topic(topic_key: str) -> str:
    """Get a specific topic by its key"""
    return DEBATE_TOPICS.get(topic_key, "Topic not found!")

# Let's see our topics!
list_topics()

# Try getting a specific topic
print(f"\n🍍 Pizza topic: {get_topic('pineapple_pizza')}")


## Part 2: Crafting AI Prompts 📝

The secret sauce of making AI agents behave the way we want is **prompt engineering** - writing clear, specific instructions that guide the AI's behavior.

Let's look at how we create prompts for our debate agents and judge:


In [ ]:
# These are the "instructions" we give to our AI agents
# Think of them like a script that tells the AI how to act!

# 🎭 Instructions for debate agents
DEBATE_AGENT_PROMPT = """You are a debate agent. Your goal is to argue your assigned stance on a given topic.
You must be persuasive, clear, and consistent in your arguments. Do not sound like a robot or over pompous. 
Be concise and to the point, use no more than 100 words or less.
You are debating against other agents. You will be given their previous statements to respond to.

Debate Topic: {topic}
Your Stance: {stance}
"""

# 🎤 Instructions for opening statements
OPENING_STATEMENT_PROMPT = """
Please provide a strong and clear opening statement based on your assigned stance. 
Do not sound like a robot/AI or over pompous. Be concise and to the point, use no more than 100 words or less.
"""

# 💬 Instructions for responding to other agents
DEBATE_RESPONSE_PROMPT = """
Here are the previous statements from other agents:
{previous_statements}

Based on your assigned stance and their arguments, provide a compelling response.
Address their points and reinforce your own position. Do not sound like a robot or over pompous. 
Be concise and to the point, use no more than 100 words or less.
"""

# ⚖️ Instructions for the judge
JUDGE_PROMPT = """You are an impartial judge in a multi-agent debate. Your task is to determine the winner based on the provided transcript.
Analyze the arguments of each agent for clarity, persuasiveness, consistency, and how well they responded to their opponents.

The debate transcript is provided below. Each entry has a round number, the agent ID, and their message.

{transcript}

After reviewing the entire debate, you must declare a winner.
Your output MUST be in the following format, and nothing else:

Winner: Agent <X>
Justification: <Your detailed justification for choosing the winner.>

Replace <X> with the agent number you have chosen as the winner.
"""

print("✅ All prompts defined!")
print("\n🔍 Let's see how a prompt looks when filled in:")

# Example: Fill in the debate agent prompt
example_prompt = DEBATE_AGENT_PROMPT.format(
    topic="Pineapple is a valid and delicious pizza topping.",
    stance="Strongly Against"
)
print("\n" + "="*50)
print("EXAMPLE DEBATE AGENT PROMPT:")
print("="*50)
print(example_prompt)


## Part 3: Creating AI Debate Agents 🤖

Now let's create our AI agents! Each agent will have a personality, stance, and the ability to respond in debates. We'll create classes to organize our code nicely.


In [ ]:
class DebateAgent:
    """
    A single AI agent that can participate in debates
    """
    
    def __init__(self, agent_id: int, model: str, stance: str, topic: str):
        self.agent_id = agent_id
        self.model = model
        self.stance = stance
        self.topic = topic
        
        # Create the system prompt for this agent
        self.system_prompt = DEBATE_AGENT_PROMPT.format(
            topic=topic,
            stance=stance
        )
        
        print(f"🤖 Created Agent {agent_id}: {stance} (using {model})")
    
    def get_opening_statement(self) -> str:
        """Get the agent's opening statement"""
        try:
            response = chat(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": OPENING_STATEMENT_PROMPT}
                ],
                options={"temperature": 0.7}
            )
            return response.message.content
        except Exception as e:
            return f"Error getting opening statement: {e}"
    
    def respond_to_debate(self, previous_statements: str) -> str:
        """Respond to other agents' statements"""
        try:
            prompt = DEBATE_RESPONSE_PROMPT.format(
                previous_statements=previous_statements
            )
            
            response = chat(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                options={"temperature": 0.7}
            )
            return response.message.content
        except Exception as e:
            return f"Error responding to debate: {e}"

class JudgeAgent:
    """
    An AI judge that determines the winner of a debate
    """
    
    def __init__(self, model: str):
        self.model = model
        print(f"⚖️ Created Judge using {model}")
    
    def judge_debate(self, transcript: str) -> Dict[str, str]:
        """Judge the debate and return winner + justification"""
        try:
            prompt = JUDGE_PROMPT.format(transcript=transcript)
            
            response = chat(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                options={"temperature": 0.3}  # Lower temperature for more consistent judging
            )
            
            # Parse the judge's response
            content = response.message.content
            
            # Look for "Winner: Agent X" pattern
            winner_match = re.search(r"Winner:\s*Agent\s*(\d+)", content)
            
            # Look for justification
            justification_match = re.search(r"Justification:\s*(.*)", content, re.DOTALL)
            
            return {
                "winner": winner_match.group(1) if winner_match else "Unknown",
                "justification": justification_match.group(1).strip() if justification_match else "No justification provided",
                "raw_response": content
            }
            
        except Exception as e:
            return {
                "winner": "Error",
                "justification": f"Error judging debate: {e}",
                "raw_response": ""
            }

print("✅ Agent classes created!")
print("\n🎯 Let's test creating an agent:")

# Test creating an agent (you'll need to have a model available)
try:
    test_agent = DebateAgent(
        agent_id=1,
        model="llama3.2",  # Change this to a model you have
        stance="Strongly For",
        topic="Pineapple is a valid and delicious pizza topping."
    )
    print("✅ Test agent created successfully!")
except Exception as e:
    print(f"⚠️ Couldn't create test agent: {e}")
    print("Make sure you have Ollama running and a model installed!")


## Part 4: Creating a Simple Debate 🎪

Now let's put it all together and create a simple 2-agent debate! We'll create two agents with opposing views and have them debate for a few rounds.


In [ ]:
def run_simple_debate(topic_key: str, model: str = "llama3.2", rounds: int = 2):
    """
    Run a simple 2-agent debate
    
    Args:
        topic_key: The key for the debate topic
        model: The AI model to use
        rounds: Number of debate rounds
    """
    
    # Get the topic
    topic = get_topic(topic_key)
    if not topic:
        print(f"❌ Topic '{topic_key}' not found!")
        return
    
    print(f"🎭 Starting debate on: {topic}")
    print("=" * 60)
    
    # Create two agents with opposing views
    agent1 = DebateAgent(1, model, "Strongly For", topic)
    agent2 = DebateAgent(2, model, "Strongly Against", topic)
    
    # Create a judge
    judge = JudgeAgent(model)
    
    print("\n🎪 THE DEBATE BEGINS!")
    print("=" * 60)
    
    # Store the debate transcript
    transcript = []
    
    # Round 0: Opening statements
    print("\n🎤 OPENING STATEMENTS:")
    print("-" * 30)
    
    # Agent 1 opens
    statement1 = agent1.get_opening_statement()
    print(f"\n🤖 Agent 1 ({agent1.stance}):")
    print(statement1)
    transcript.append(f"Round 0 - Agent 1: {statement1}")
    
    # Agent 2 opens  
    statement2 = agent2.get_opening_statement()
    print(f"\n🤖 Agent 2 ({agent2.stance}):")
    print(statement2)
    transcript.append(f"Round 0 - Agent 2: {statement2}")
    
    # Debate rounds
    for round_num in range(1, rounds + 1):
        print(f"\n🔥 ROUND {round_num}:")
        print("-" * 20)
        
        # Prepare previous statements for context
        previous_statements = "\\n".join(transcript[-2:])  # Last 2 statements
        
        # Agent 1 responds
        response1 = agent1.respond_to_debate(previous_statements)
        print(f"\n🤖 Agent 1:")
        print(response1)
        transcript.append(f"Round {round_num} - Agent 1: {response1}")
        
        # Agent 2 responds
        previous_statements = "\\n".join(transcript[-2:])
        response2 = agent2.respond_to_debate(previous_statements)
        print(f"\n🤖 Agent 2:")
        print(response2)
        transcript.append(f"Round {round_num} - Agent 2: {response2}")
    
    # Judge the debate
    print("\n⚖️ JUDGING...")
    print("-" * 20)
    
    full_transcript = "\\n\\n".join(transcript)
    judgment = judge.judge_debate(full_transcript)
    
    print(f"\n🏆 WINNER: Agent {judgment['winner']}")
    print(f"\n📝 JUSTIFICATION:")
    print(judgment['justification'])
    
    return {
        "topic": topic,
        "transcript": transcript,
        "winner": judgment['winner'],
        "justification": judgment['justification']
    }

# Let's run a debate!
print("🚀 Running a sample debate...")
print("Note: Make sure you have a model installed in Ollama!")

try:
    debate_result = run_simple_debate("pineapple_pizza", "llama3.2", rounds=1)
    print("\\n✅ Debate completed!")
except Exception as e:
    print(f"❌ Error running debate: {e}")
    print("Make sure Ollama is running and you have the model installed!")


## Part 5: Saving and Analyzing Debates 💾

A real project needs to save its results! Let's create functions to save our debates and analyze the results. This is important for keeping track of what our AI agents are doing.


In [ ]:
def save_debate_results(debate_result: dict, results_dir: str = "debate_results"):
    """
    Save debate results to files
    
    Args:
        debate_result: The result from run_simple_debate()
        results_dir: Directory to save results
    """
    
    # Create results directory if it doesn't exist
    os.makedirs(results_dir, exist_ok=True)
    
    # Create a timestamp for unique folder names
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_name = f"debate_{timestamp}"
    debate_folder = os.path.join(results_dir, folder_name)
    os.makedirs(debate_folder, exist_ok=True)
    
    # Save transcript as JSONL (one JSON object per line)
    transcript_file = os.path.join(debate_folder, "transcript.jsonl")
    with open(transcript_file, "w") as f:
        for i, statement in enumerate(debate_result["transcript"]):
            event = {
                "event_type": "statement",
                "timestamp": datetime.now().isoformat(),
                "content": statement
            }
            f.write(json.dumps(event) + "\\n")
        
        # Add the final judgment
        judgment_event = {
            "event_type": "judgment",
            "timestamp": datetime.now().isoformat(),
            "winner": debate_result["winner"],
            "justification": debate_result["justification"]
        }
        f.write(json.dumps(judgment_event) + "\\n")
    
    # Save metadata
    metadata_file = os.path.join(debate_folder, "metadata.json")
    metadata = {
        "topic": debate_result["topic"],
        "winner": debate_result["winner"],
        "timestamp": timestamp,
        "total_statements": len(debate_result["transcript"])
    }
    
    with open(metadata_file, "w") as f:
        json.dump(metadata, f, indent=2)
    
    print(f"💾 Debate saved to: {debate_folder}")
    return debate_folder

def create_simple_visualization(debate_result: dict):
    """
    Create a simple text-based visualization of the debate
    """
    print("\\n" + "="*80)
    print("🎭 DEBATE VISUALIZATION")
    print("="*80)
    
    print(f"\\n📋 Topic: {debate_result['topic']}")
    print(f"🏆 Winner: Agent {debate_result['winner']}")
    print(f"📊 Total Statements: {len(debate_result['transcript'])}")
    
    print("\\n💬 DEBATE FLOW:")
    print("-" * 40)
    
    for i, statement in enumerate(debate_result['transcript']):
        # Extract round and agent info
        parts = statement.split(': ', 1)
        if len(parts) == 2:
            round_agent, content = parts
            
            # Add some visual formatting
            if 'Agent 1' in round_agent:
                print(f"\\n🟦 {round_agent}:")
            else:
                print(f"\\n🟥 {round_agent}:")
            
            # Truncate long statements for overview
            if len(content) > 100:
                content = content[:100] + "..."
            print(f"   {content}")
    
    print("\\n" + "="*80)

# Test our saving and visualization functions
print("💾 Testing save and visualization functions...")

# Note: This will only work if you successfully ran a debate above
try:
    if 'debate_result' in locals():
        # Save the debate
        saved_folder = save_debate_results(debate_result)
        
        # Create visualization
        create_simple_visualization(debate_result)
        
        print(f"\\n✅ Results saved and visualized!")
    else:
        print("⚠️ No debate result to save. Run a debate first!")
        
except Exception as e:
    print(f"❌ Error saving/visualizing: {e}")


## Part 6: Try It Yourself! 🎯

Now it's your turn to experiment! Here are some fun things you can try:

### 🧪 Experiments to Try:

1. **Different Topics**: Try debates on different topics from our list
2. **Different Stances**: Create agents with different viewpoints (Moderate, Skeptical, etc.)
3. **More Rounds**: Run longer debates to see how arguments develop
4. **Different Models**: If you have multiple models, try different combinations
5. **Custom Topics**: Add your own topics to the DEBATE_TOPICS dictionary

### 💡 Prompt Engineering Challenges:

1. **Personality**: Modify the agent prompts to give them different personalities
2. **Expertise**: Make agents experts in specific fields
3. **Speaking Style**: Make agents speak like different types of people (scientist, teenager, etc.)

Let's create an interactive function for you to experiment:


In [ ]:
def interactive_debate_setup():
    """
    Interactive function to set up and run custom debates
    """
    print("🎭 Welcome to the Interactive Debate Creator!")
    print("=" * 50)
    
    # Show available topics
    list_topics()
    
    # Get user choices
    print("\\n🎯 Let's set up your debate:")
    
    # Choose topic
    topic_keys = list(DEBATE_TOPICS.keys())
    print(f"\\nAvailable topics: {', '.join(topic_keys)}")
    topic_choice = input("Enter topic key (or 'custom' for your own): ").strip()
    
    if topic_choice == 'custom':
        custom_topic = input("Enter your custom topic: ").strip()
        DEBATE_TOPICS['custom'] = custom_topic
        topic_choice = 'custom'
    
    # Choose model (show available models if possible)
    try:
        models = ollama_list()
        available_models = [m['model'] for m in models['models']]
        print(f"\\nAvailable models: {', '.join(available_models[:5])}...")  # Show first 5
        model_choice = input("Enter model name: ").strip()
    except:
        model_choice = input("Enter model name (e.g., llama3.2): ").strip()
    
    # Choose number of rounds
    try:
        rounds = int(input("Number of debate rounds (1-5): "))
        rounds = max(1, min(5, rounds))  # Limit between 1-5
    except:
        rounds = 2
    
    # Choose agent stances
    print("\\nChoose stances for your agents:")
    stance1 = input("Agent 1 stance (e.g., 'Strongly For', 'Moderately Against'): ").strip()
    stance2 = input("Agent 2 stance (e.g., 'Strongly Against', 'Skeptical'): ").strip()
    
    print(f"\\n🚀 Starting debate with:")
    print(f"   Topic: {get_topic(topic_choice)}")
    print(f"   Model: {model_choice}")
    print(f"   Rounds: {rounds}")
    print(f"   Agent 1: {stance1}")
    print(f"   Agent 2: {stance2}")
    
    confirm = input("\\nProceed? (y/n): ").lower().startswith('y')
    
    if confirm:
        try:
            # Run custom debate
            result = run_custom_debate(topic_choice, model_choice, stance1, stance2, rounds)
            
            # Save and visualize
            if result:
                save_debate_results(result)
                create_simple_visualization(result)
            
        except Exception as e:
            print(f"❌ Error running debate: {e}")
    else:
        print("👋 Maybe next time!")

def run_custom_debate(topic_key: str, model: str, stance1: str, stance2: str, rounds: int):
    """Run a custom debate with specified parameters"""
    
    topic = get_topic(topic_key)
    if not topic:
        print(f"❌ Topic '{topic_key}' not found!")
        return None
    
    print(f"\\n🎭 Custom Debate: {topic}")
    print("=" * 60)
    
    # Create agents with custom stances
    agent1 = DebateAgent(1, model, stance1, topic)
    agent2 = DebateAgent(2, model, stance2, topic)
    judge = JudgeAgent(model)
    
    transcript = []
    
    # Opening statements
    print("\\n🎤 OPENING STATEMENTS:")
    
    statement1 = agent1.get_opening_statement()
    print(f"\\n🟦 Agent 1 ({stance1}):")
    print(statement1)
    transcript.append(f"Round 0 - Agent 1: {statement1}")
    
    statement2 = agent2.get_opening_statement()
    print(f"\\n🟥 Agent 2 ({stance2}):")
    print(statement2)
    transcript.append(f"Round 0 - Agent 2: {statement2}")
    
    # Debate rounds
    for round_num in range(1, rounds + 1):
        print(f"\\n🔥 ROUND {round_num}:")
        
        previous_statements = "\\n".join(transcript[-2:])
        
        response1 = agent1.respond_to_debate(previous_statements)
        print(f"\\n🟦 Agent 1:")
        print(response1)
        transcript.append(f"Round {round_num} - Agent 1: {response1}")
        
        previous_statements = "\\n".join(transcript[-2:])
        response2 = agent2.respond_to_debate(previous_statements)
        print(f"\\n🟥 Agent 2:")
        print(response2)
        transcript.append(f"Round {round_num} - Agent 2: {response2}")
    
    # Judge
    print("\\n⚖️ JUDGING...")
    full_transcript = "\\n\\n".join(transcript)
    judgment = judge.judge_debate(full_transcript)
    
    print(f"\\n🏆 WINNER: Agent {judgment['winner']}")
    print(f"\\n📝 JUSTIFICATION: {judgment['justification']}")
    
    return {
        "topic": topic,
        "transcript": transcript,
        "winner": judgment['winner'],
        "justification": judgment['justification']
    }

# Uncomment the line below to run the interactive setup
# interactive_debate_setup()

print("✅ Interactive debate functions ready!")
print("\\n🎯 To start an interactive debate, run: interactive_debate_setup()")
print("\\n💡 Or modify the examples above to try different configurations!")


## What You've Learned! 🎓

Congratulations! You've just built a complete multi-agent AI debate system. Here's what you've learned:

### 🧠 Key Concepts:

1. **Agentic AI**: You created AI agents with specific roles, personalities, and goals
2. **Prompt Engineering**: You learned how to write instructions that guide AI behavior
3. **Multi-Agent Systems**: You saw how multiple AI agents can interact and compete
4. **Data Management**: You learned to save, organize, and analyze AI interactions
5. **Real Project Structure**: You built a complete, working AI project

### 🛠️ Technical Skills:

- Working with the Ollama API
- Creating Python classes for AI agents
- Managing conversation history and context
- Parsing and validating AI responses
- Saving structured data (JSON, JSONL)
- Building interactive interfaces

### 🚀 Next Steps:

1. **Experiment**: Try different topics, models, and agent personalities
2. **Extend**: Add more agent types (Moderator, Fact-checker, etc.)
3. **Improve**: Add better error handling and user interfaces
4. **Scale**: Try debates with 3+ agents
5. **Analyze**: Create visualizations and statistics about debate outcomes

### 💡 Project Ideas:

- **Educational Debates**: Create debates for learning topics at school
- **Decision Making**: Use debates to explore pros/cons of decisions
- **Creative Writing**: Have agents debate story directions
- **Research**: Explore different perspectives on complex topics

You now have the foundation to build sophisticated AI applications! Keep experimenting and learning! 🌟
